In [9]:
# Imports & Inits

from datetime import datetime
from datetime import timezone
import polars as pl
import os

from utils import two_point_calibration, average_bottle, plot_sensor_measurement

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORY = os.environ.get("PICARRO_DATA_DIRECTORY")


#------------
sensor_id = 14
#------------

df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet"))

df = df.filter(pl.col("system_name") == f"tum-esm-midcost-raspi-{sensor_id}") \
    .sort(pl.col("creation_timestamp")) \
    .filter(pl.col("gmp343_filtered") > 0.0) \
    .select(pl.col("creation_timestamp"),
                pl.col("gmp343_filtered"),
                pl.col("sht45_humidity"))

In [10]:
#600
start_date = datetime(2023, 11, 16, 15, 32, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 11, 16, 16, 2, 0).replace(tzinfo=timezone.utc)

df_p_600 = df.sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date))

plot_sensor_measurement(df_p_600.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_600.collect(), col_name="sht45_humidity")

#400
start_date = datetime(2023, 11, 16, 16, 4, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 11, 16, 16, 34, 0).replace(tzinfo=timezone.utc)

df_p_400 = df.filter(pl.col("creation_timestamp").is_between(start_date, end_date))

plot_sensor_measurement(df_p_400.collect(), col_name="gmp343_filtered")
plot_sensor_measurement(df_p_400.collect(), col_name="sht45_humidity")

In [11]:
measured_values = [None, None]
true_values = [427.38, 610.95]

# 400 ppm

data = df_p_400.select(pl.col("gmp343_filtered")).collect().to_series().to_list()
measured_values[0] = average_bottle(data)

# 600 ppm
data = df_p_600.select(pl.col("gmp343_filtered")).collect().to_series().to_list()

measured_values[1] = average_bottle(data)

# calculate slope and intercept
slope, intercept = two_point_calibration(measured_values, true_values)
print(f"acropolis_slope = {slope}, acropolis_intercept = {intercept}")

Uncut average: 433.6782857142856
Cut average: 433.2798245614035
Uncut average: 614.6805555555555
Cut average: 614.8034188034187
acropolis_slope = 1.0112734973463364, acropolis_intercept = -10.784403513817551
